Import necessary dependencies:

In [44]:
import boto3
import csv
import os
import logging
import argparse
import json
import psycopg2
from psycopg2.extras import RealDictCursor
from os import environ
from dotenv import load_dotenv

Create connection to database:

In [45]:
def get_connection():
    """Returns connection"""
    return psycopg2.connect(
        dbname=environ.get("DB_NAME"),
        host=environ.get("HOST"),
        user=environ.get("USERNAME"),
        password=environ.get("PASSWORD")
    )


def get_cursor(conn):
    """Returns cursor object"""
    return conn.cursor(cursor_factory=RealDictCursor)

load_dotenv()
conn = get_connection()
cursor = get_cursor(conn)

SQL Queries:

Most frequently visited exhibition:

In [46]:
cursor.execute("""
    SELECT exhibition_name, count(rating_id) as count_ratings from exhibition as e
    join rating_interaction as ri on ri.exhibition_id = e.exhibition_id
    group by exhibition_name
    order by count_ratings DESC
    limit 1;
""")
conn.commit()
most_visited = cursor.fetchall()

for row in most_visited:
    print(f"Exhibition: {row['exhibition_name']}")
    print(f"Number of rating interactions: {row['count_ratings']}")

Exhibition: Our Polluted World
Number of rating interactions: 481


Hour of day with most ratings:

In [47]:
cursor.execute("""
    select EXTRACT(HOUR FROM event_at) as hour, count(rating_interaction_id) as num_ratings
    from rating_interaction
    group by hour
    order by num_ratings DESC
    limit 1;
""")
conn.commit()
busy_hour = cursor.fetchone()

print(f"Busiest hour: {busy_hour['hour']}")
print(f"Number of ratings during this hour: {busy_hour['num_ratings']}")

Busiest hour: 10
Number of ratings during this hour: 232


What exhibition has the most emergencies?

In [48]:
cursor.execute("""
    select count(request_interaction_id) as num_emg, e.exhibition_name
    from request_interaction as ri
    join exhibition as e on e.exhibition_id = ri.exhibition_id
    where ri.request_id = 1.0
    group by e.exhibition_name
    order by num_emg;
""")
conn.commit()
exhib_emg = cursor.fetchone()

print(f"Exhibition name: {exhib_emg['exhibition_name']}")
print(f"Number of emergencies: {exhib_emg['num_emg']}")

Exhibition name: The Crenshaw Collection
Number of emergencies: 2


What is the average rating for each exhibition?

In [49]:
cursor.execute("""
    select AVG(ri.rating_id) as avg_rate, exhibition_name
    from rating_interaction as ri
    join exhibition as e on ri.exhibition_id = e.exhibition_id
    group by exhibition_name
    order by avg_rate DESC;
""")

conn.commit()
avg_ratings = cursor.fetchall()

for row in avg_ratings:
    print(f"Exhibition: {row['exhibition_name']}, AVG rating: {row['avg_rate']}")

Exhibition: Cetacean Sensations, AVG rating: 2.8305084745762712
Exhibition: Adaptation, AVG rating: 1.9294117647058824
Exhibition: Measureless to Man, AVG rating: 1.9169435215946844
Exhibition: The Crenshaw Collection, AVG rating: 1.4259740259740260
Exhibition: Our Polluted World, AVG rating: 1.2162162162162162


Are positive ratings more frequent before or after 1pm? Answer: After 1pm (344 positive ratings)

In [50]:
cursor.execute("""
with ratings_ph as (
    select count(rating_interaction_id) as pos_rate, EXTRACT(HOUR FROM event_at) as hour
    from rating_interaction as ri
    where ri.rating_id > 2
    group by hour
)
select SUM(pos_rate)
from ratings_ph as rph
where rph.hour >= 13;
""")

conn.commit()
num_pos_rate = cursor.fetchone()

print(f"Number of positive ratings after 1pm: {num_pos_rate['sum']}")


Number of positive ratings after 1pm: 344


Do Zoology exhibitions get better ratings than other types? Answer: Yes, yes they do.

In [51]:
cursor.execute("""select AVG(ri.rating_id) as avg_rate, department_name
from rating_interaction as ri
join exhibition as e on ri.exhibition_id = e.exhibition_id
join department as d on d.department_id = e.department_id
group by department_name
order by avg_rate DESC;""")

conn.commit()
depart_ratings = cursor.fetchall()

for row in depart_ratings:
    print(f"Department: {row['department_name']}, AVG rating: {row['avg_rate']}")

Department: Zoology, AVG rating: 2.1995332555425904
Department: Entomology, AVG rating: 1.9294117647058824
Department: Geology, AVG rating: 1.9169435215946844
Department: Ecology, AVG rating: 1.2162162162162162
